In [2]:
import cv2 
import numpy as np
import matplotlib.pyplot as mpl 
from matplotlib import cm

### Primeiramente, foi realizado uma função apenas para ver o efeito do método de Floyd e Steinberg para analisar os efeitos desse tipo de abordagem em uma imagem lida com tons de cinza.

In [2]:
def FloydSteinberg(f):
    height, width = f.shape
    g = np.ones((height, width), dtype = "float32")
    f = cv2.normalize(f, None, alpha=0, beta=1, norm_type= cv2.NORM_MINMAX, dtype= cv2.CV_32F)
    for y in range(0, height - 1):
        for x in range(1, width - 1):
            if f[y, x] < 0.5:
                g[y, x] = 0
            else:
                g[y, x] = 1
            erro = f[y, x] - g[y, x]
            f[y, x+1] = f[y, x+1] + (7/16)*erro 
            f[y+1, x-1] = f[y+1, x-1] + (3/16)*erro
            f[y+1, x] = f[y+1, x] + (5/16)*erro
            f[y+1, x+1] =  f[y+1, x+1] + (1/16)*erro
    return np.uint8(g)


In [3]:
img = cv2.imread("baboon.png", 0) 
im2 = FloydSteinberg(img)
cv2.imshow("baboon", im2*255)

cv2.waitKey(0)
cv2.destroyAllWindows()


### Então em seguida, foi adotado um sentido diferente ao linear, dessa vez passando em sentido alternado pelas linhas da imagem 

In [4]:

def FloydSteinbergsentido(f):
    height, width= f.shape
    g = np.ones((height, width), dtype = "float32")
    f = cv2.normalize(f, None, alpha=0, beta=1, norm_type= cv2.NORM_MINMAX, dtype= cv2.CV_32F)
    for x in range(1, width - 1):
        if x&1:
            for y in range(0, height - 1):          
                if f[y, x] < 0.5:
                    g[y, x] = 0
                else:
                    g[y, x] = 1
                erro = f[ y, x] - g[ y, x]
                f[y+1, x] = f[y+1, x] + (7/16)*erro 
                f[y-1, x+1] = f[y-1, x+1] + (3/16)*erro
                f[y, x+1] = f[y, x+1] + (5/16)*erro
                f[y+1, x+1] =  f[y+1, x+1] + (1/16)*erro
        else:
            for y in range(height - 2, 0, -1):
                if f[y, x] < 0.5:
                    g[y, x] = 0
                else:
                    g[y, x] = 1
                erro = f[y, x] - g[y, x]
                f[y-1, x] = f[y-1, x] + (7/16)*erro 
                f[y-1, x+1] = f[y-1, x+1] + (1/16)*erro
                f[y, x+1] = f[y, x+1] + (5/16)*erro
                f[y+1, x+1] =  f[y+1, x+1] + (3/16)*erro
    return np.uint8(g)




In [5]:
img = cv2.imread("baboon.png", 0) 
im2 = FloydSteinbergsentido(img)
cv2.imshow("baboon", im2*255)

cv2.waitKey(0)
cv2.destroyAllWindows()


In [6]:
def FloydSteinbergcolor(f):
    height, width, channels = f.shape
    g = np.ones((height, width, channels), dtype = "float32")
    f = cv2.normalize(f, None, alpha=0, beta=1, norm_type= cv2.NORM_MINMAX, dtype= cv2.CV_32F)
    
    for z in range(channels):
        for x in range(1, width - 1):
            if x&1:
                for y in range(0, height - 1):          
                    if f[y, x, z] < 0.5:
                        g[y, x, z] = 0
                    else:
                        g[y, x, z] = 1
                    erro = f[ y, x, z] - g[ y, x, z]
                    f[y+1, x,  z] = f[y+1, x, z] + (7/16)*erro 
                    f[y-1, x+1,  z] = f[y-1, x+1, z] + (3/16)*erro
                    f[y, x+1, z] = f[y, x+1, z] + (5/16)*erro
                    f[y+1, x+1, z] =  f[y+1, x+1, z] + (1/16)*erro
            else:
                for y in range(height - 2, 0, -1):
                    if f[y, x, z] < 0.5:
                        g[y, x, z] = 0
                    else:
                        g[y, x, z] = 1
                    erro = f[y, x, z] - g[y, x, z]
                    f[y-1, x, z] = f[y-1, x, z] + (7/16)*erro 
                    f[y-1, x+1, z] = f[y-1, x+1, z] + (1/16)*erro
                    f[y, x+1, z] = f[y, x+1, z] + (5/16)*erro
                    f[y+1, x+1,  z] =  f[y+1, x+1, z] + (3/16)*erro
    return np.uint8(g)

In [7]:
img = cv2.imread("baboon.png") 
im2 = FloydSteinbergcolor(img)
cv2.imshow("baboon", im2*255)

cv2.waitKey(0)
cv2.destroyAllWindows()


### Teste agora então de uma adição de um modo de passar matrizes na função, para implementar os outros fiitros.

In [2]:
def dithering(f, n):
    height, width = f.shape
    g = np.ones((height, width), dtype = "float32")
    f = cv2.normalize(f, None, alpha=0, beta=1, norm_type= cv2.NORM_MINMAX, dtype= cv2.CV_32F)
    pady = 0
    padx = 0
    
    if n == "a":
        a = np.array([[0, 0, 7/16], [3/16, 5/16, 1/16]])
    if n == "b":
        a = np.array([[0,0,0,0,0,32/200,0], 
                           [12/200,0,26/200,0,30/200,0,16/200],
                           [0,12/200,0,26/200,0,12/200,0],
                           [5/200,0,12/200,0,12/200,0,5/100]])      
    if n == "c":
        a = np.array([[0, 0, 0, 8/32, 4/32], 
                      [2/32, 4/32, 8/32, 4/32, 2/32 ]])
    if n == "d":
        a = np.array([[0, 0, 0, 5/32, 3/32], 
                      [2/32, 4/32, 5/32, 4/32, 2/32],
                      [0, 2/32, 3/32, 2/32, 0]])
    if n == "e":
        a = np.array([[0, 0, 0, 8/42, 4/42], 
                      [2/42, 4/42, 8/42, 4/42, 2/42],
                      [1/42, 2/42, 4/42, 2/42, 1/42]])
    if n == "f":
        a = np.array([[0, 0, 0, 7/48, 5/48], 
                      [3/48, 5/48, 7/48, 5/48, 3/48],
                      [1/48, 3/48, 5/48, 3/48, 1/48]])
    pady = a.shape[0] - 1
    padx = a.shape[1] >> 1
    for y in range(0, height - pady):
        for x in range(padx, width - padx):
            if f[y, x] < 0.5:
                g[y, x] = 0
            else:
                g[y, x] = 1
            erro = f[y, x] - g[y, x]
            for i in range(a.shape[0]):
                for j in range(a.shape[1]):
                    f[(y+i) , (x-padx+j)] = f[(y+i) , (x-padx+j)] + a[i, j] * erro 
    return np.uint8(g)


In [ ]:
img = cv2.imread("baboon.png", 0) 
im2 = dithering(img, "f")
cv2.imshow("baboon", im2*255)

cv2.waitKey(0)
cv2.destroyAllWindows()


Agora implementamos o sentido de zigzag.

In [3]:
def ditheringzigzag(f, n):
    height, width = f.shape
    g = np.ones((height, width), dtype = "float32")
    f = cv2.normalize(f, None, alpha=0, beta=1, norm_type= cv2.NORM_MINMAX, dtype= cv2.CV_32F)
    pady = 0
    padx = 0
    
    if n == "a":
        a = np.array([[0, 0, 7/16], [3/16, 5/16, 1/16]])
        b = np.array([[7/16, 0, 0], [1/16, 5/16, 3/16]])
    if n == "b":
        a = np.array([[0,0,0,0,0,32/200,0], 
                           [12/200,0,26/200,0,30/200,0,16/200],
                           [0,12/200,0,26/200,0,12/200,0],
                           [5/200,0,12/200,0,12/200,0,5/100]])      
        b = np.array([[0,32/200,0,0,0,0,0], 
                           [16/200,0,30/200,0,26/200,0,12/200],
                           [0,12/200,0,26/200,0,12/200,0],
                           [5/200,0,12/200,0,12/200,0,5/100]])      
    
    if n == "c":
        a = np.array([[0, 0, 0, 8/32, 4/32], 
                      [2/32, 4/32, 8/32, 4/32, 2/32 ]])
        b = np.array([[4/32, 8/32, 0, 0, 0], 
                      [2/32, 4/32, 8/32, 4/32, 2/32 ]])
    if n == "d":
        a = np.array([[0, 0, 0, 5/32, 3/32], 
                      [2/32, 4/32, 5/32, 4/32, 2/32],
                      [0, 2/32, 3/32, 2/32, 0]])
        b = np.array([[3/32, 5/32, 0, 0, 0], 
                      [2/32, 4/32, 5/32, 4/32, 2/32],
                      [0, 2/32, 3/32, 2/32, 0]])
    if n == "e":
        a = np.array([[0, 0, 0, 8/42, 4/42], 
                      [2/42, 4/42, 8/42, 4/42, 2/42],
                      [1/42, 2/42, 4/42, 2/42, 1/42]])
        b = np.array([[4/42, 8/42, 0, 0, 0], 
                      [2/42, 4/42, 8/42, 4/42, 2/42],
                      [1/42, 2/42, 4/42, 2/42, 1/42]])
    if n == "f":
        a = np.array([[0, 0, 0, 7/48, 5/48], 
                      [3/48, 5/48, 7/48, 5/48, 3/48],
                      [1/48, 3/48, 5/48, 3/48, 1/48]])
        b = np.array([[5/48, 7/48, 0, 0, 0], 
                      [3/48, 5/48, 7/48, 5/48, 3/48],
                      [1/48, 3/48, 5/48, 3/48, 1/48]])
    pady = a.shape[0] - 1
    padx = a.shape[1] >> 1
    for x in range(padx, width - padx):
        if x&1:
            for y in range(0, height - pady):
                if f[y, x] < 0.5:
                    g[y, x] = 0
                else:
                    g[y, x] = 1
                erro = f[y, x] - g[y, x]
                for i in range(a.shape[0]):
                    for j in range(a.shape[1]):
                        f[(y+i) , (x-padx+j)] = f[(y+i) , (x-padx+j)] + a[i, j] * erro 
        else:
            for y in range(height - pady -1 , 0, -1):
                if f[y, x] < 0.5:
                    g[y, x] = 0
                else:
                    g[y, x] = 1
                erro = f[y, x] - g[y, x]
                for i in range(a.shape[0]):
                    for j in range(a.shape[1]):
                        f[(y+i) , (x-padx+j)] = f[(y+i) , (x-padx+j)] + b[i, j] * erro 
    return np.uint8(g)



In [ ]:
img = cv2.imread("baboon.png", 0) 
im2 = ditheringzigzag(img, "f")
im1 = dithering(img, "f")
cv2.imshow("baboon", im2*255)
cv2.imshow("baboon2", im1*255)

cv2.waitKey(0)
cv2.destroyAllWindows()

Agora resta o efeito para fotos coloridas.


In [4]:
def ditheringcolor(f, n):
    height, width, channels = f.shape
    g = np.ones((height, width, channels), dtype = "float32")
    f = cv2.normalize(f, None, alpha=0, beta=1, norm_type= cv2.NORM_MINMAX, dtype= cv2.CV_32F)
    pady = 0
    padx = 0
    
    if n == "a":
        a = np.array([[0, 0, 7/16], [3/16, 5/16, 1/16]])
        b = np.array([[7/16, 0, 0], [1/16, 5/16, 3/16]])
    if n == "b":
        a = np.array([[0,0,0,0,0,32/200,0], 
                           [12/200,0,26/200,0,30/200,0,16/200],
                           [0,12/200,0,26/200,0,12/200,0],
                           [5/200,0,12/200,0,12/200,0,5/100]])      
        b = np.array([[0,32/200,0,0,0,0,0], 
                           [16/200,0,30/200,0,26/200,0,12/200],
                           [0,12/200,0,26/200,0,12/200,0],
                           [5/200,0,12/200,0,12/200,0,5/100]])      
    
    if n == "c":
        a = np.array([[0, 0, 0, 8/32, 4/32], 
                      [2/32, 4/32, 8/32, 4/32, 2/32 ]])
        b = np.array([[4/32, 8/32, 0, 0, 0], 
                      [2/32, 4/32, 8/32, 4/32, 2/32 ]])
    if n == "d":
        a = np.array([[0, 0, 0, 5/32, 3/32], 
                      [2/32, 4/32, 5/32, 4/32, 2/32],
                      [0, 2/32, 3/32, 2/32, 0]])
        b = np.array([[3/32, 5/32, 0, 0, 0], 
                      [2/32, 4/32, 5/32, 4/32, 2/32],
                      [0, 2/32, 3/32, 2/32, 0]])
    if n == "e":
        a = np.array([[0, 0, 0, 8/42, 4/42], 
                      [2/42, 4/42, 8/42, 4/42, 2/42],
                      [1/42, 2/42, 4/42, 2/42, 1/42]])
        b = np.array([[4/42, 8/42, 0, 0, 0], 
                      [2/42, 4/42, 8/42, 4/42, 2/42],
                      [1/42, 2/42, 4/42, 2/42, 1/42]])
    if n == "f":
        a = np.array([[0, 0, 0, 7/48, 5/48], 
                      [3/48, 5/48, 7/48, 5/48, 3/48],
                      [1/48, 3/48, 5/48, 3/48, 1/48]])
        b = np.array([[5/48, 7/48, 0, 0, 0], 
                      [3/48, 5/48, 7/48, 5/48, 3/48],
                      [1/48, 3/48, 5/48, 3/48, 1/48]])
    pady = a.shape[0] - 1
    padx = a.shape[1] >> 1
    for z in range(channels):
        for x in range(padx, width - padx):
            for y in range(0, height - pady):
                if f[y, x, z] < 0.5:
                    g[y, x, z] = 0
                else:
                    g[y, x, z] = 1
                erro = f[y, x, z] - g[y, x, z]
                for i in range(a.shape[0]):
                    for j in range(a.shape[1]):
                        f[(y+i) , (x-padx+j), z] = f[(y+i) , (x-padx+j), z] + a[i, j] * erro 
    return np.uint8(g)


In [5]:
def ditheringzigzagcolor(f, n):
    height, width, channels = f.shape
    g = np.ones((height, width, channels), dtype = "float32")
    f = cv2.normalize(f, None, alpha=0, beta=1, norm_type= cv2.NORM_MINMAX, dtype= cv2.CV_32F)
    pady = 0
    padx = 0
    
    if n == "a":
        a = np.array([[0, 0, 7/16], [3/16, 5/16, 1/16]])
        b = np.array([[7/16, 0, 0], [1/16, 5/16, 3/16]])
    if n == "b":
        a = np.array([[0,0,0,0,0,32/200,0], 
                           [12/200,0,26/200,0,30/200,0,16/200],
                           [0,12/200,0,26/200,0,12/200,0],
                           [5/200,0,12/200,0,12/200,0,5/100]])      
        b = np.array([[0,32/200,0,0,0,0,0], 
                           [16/200,0,30/200,0,26/200,0,12/200],
                           [0,12/200,0,26/200,0,12/200,0],
                           [5/200,0,12/200,0,12/200,0,5/100]])      
    
    if n == "c":
        a = np.array([[0, 0, 0, 8/32, 4/32], 
                      [2/32, 4/32, 8/32, 4/32, 2/32 ]])
        b = np.array([[4/32, 8/32, 0, 0, 0], 
                      [2/32, 4/32, 8/32, 4/32, 2/32 ]])
    if n == "d":
        a = np.array([[0, 0, 0, 5/32, 3/32], 
                      [2/32, 4/32, 5/32, 4/32, 2/32],
                      [0, 2/32, 3/32, 2/32, 0]])
        b = np.array([[3/32, 5/32, 0, 0, 0], 
                      [2/32, 4/32, 5/32, 4/32, 2/32],
                      [0, 2/32, 3/32, 2/32, 0]])
    if n == "e":
        a = np.array([[0, 0, 0, 8/42, 4/42], 
                      [2/42, 4/42, 8/42, 4/42, 2/42],
                      [1/42, 2/42, 4/42, 2/42, 1/42]])
        b = np.array([[4/42, 8/42, 0, 0, 0], 
                      [2/42, 4/42, 8/42, 4/42, 2/42],
                      [1/42, 2/42, 4/42, 2/42, 1/42]])
    if n == "f":
        a = np.array([[0, 0, 0, 7/48, 5/48], 
                      [3/48, 5/48, 7/48, 5/48, 3/48],
                      [1/48, 3/48, 5/48, 3/48, 1/48]])
        b = np.array([[5/48, 7/48, 0, 0, 0], 
                      [3/48, 5/48, 7/48, 5/48, 3/48],
                      [1/48, 3/48, 5/48, 3/48, 1/48]])
    pady = a.shape[0] - 1
    padx = a.shape[1] >> 1
    for z in range(channels):
        for x in range(padx, width - padx):
            if x&1:
                for y in range(0, height - pady):
                    if f[y, x, z] < 0.5:
                        g[y, x, z] = 0
                    else:
                        g[y, x, z] = 1
                    erro = f[y, x, z] - g[y, x, z]
                    for i in range(a.shape[0]):
                        for j in range(a.shape[1]):
                            f[(y+i) , (x-padx+j), z] = f[(y+i) , (x-padx+j), z] + a[i, j] * erro 
            else:
                for y in range(height - pady -1 , 0, -1):
                    if f[y, x, z] < 0.5:
                        g[y, x, z] = 0
                    else:
                        g[y, x, z] = 1
                    erro = f[y, x, z] - g[y, x, z]
                    for i in range(a.shape[0]):
                        for j in range(a.shape[1]):
                            f[(y+i) , (x-padx+j), z] = f[(y+i) , (x-padx+j), z] + b[i, j] * erro 
    return np.uint8(g)


In [5]:
img = cv2.imread("baboon.png") 
im1 = ditheringcolor(img, "f")

im2 = ditheringzigzagcolor(img, "f")

cv2.imshow("babooncolor", im1*255)
cv2.imshow("baboonzigzagcolor", im2*255)
cv2.imshow("difference", (im2*255 - im1*255))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [7]:
img = cv2.imread("amor.jpeg", 0) 
im1 = ditheringzigzag(img, "d")
cv2.imwrite("amorpont.jpeg", im1*255)


True

In [ ]:
imggray = cv2.imread("baboon.png", 0) 
img = cv2.imread("baboon.png") 
im1 = ditheringcolor(img, "b")
im2 = ditheringzigzagcolor(img, "b")
im3 = dithering(imggray, "b")
im4 = ditheringzigzag(imggray, "b")

cv2.imwrite("baboonditb.png", im3*255)
cv2.imwrite("baboonzigzagditb.png", im4*255)
cv2.imwrite("differencegrayb.png", (im4*255 - im3*255))

cv2.imwrite("babooncolorb.png", im1*255)
cv2.imwrite("baboonzigzagcolorb.png", im2*255)
cv2.imwrite("differencecolorb.png", (im2*255 - im1*255))

In [5]:
f = cv2.imread("baboon.png", 0) 
g = np.array(f, copy=True)
out = np.where(g>=127,255,0)
cv2.imwrite("tryed.png", out)


In [ ]:
x = cv2.imread("tryed.png", 0)
cv2.imshow("baboon", x)
cv2.waitKey(0)
cv2.destroyAllWindows()